In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

In [3]:
from keras.models import Sequential
from keras import Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout,Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras_vggface.vggface import VGGFace
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import Adam
import keras
import pandas
import ast


Using TensorFlow backend.


In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [5]:
train_set = ".\\dataset\\NUOVO\\train"
validation_set = ".\\dataset\\NUOVO\\val"
test_set = ".\\dataset\\NUOVO\\test"

In [6]:
from keras.preprocessing.image import load_img, ImageDataGenerator

In [7]:
train_df = pandas.read_csv(".\\dataset\\NUOVO\\train\\output.csv")
valid_df = pandas.read_csv(".\\dataset\\NUOVO\\val\\output.csv")
test_df = pandas.read_csv(".\\dataset\\NUOVO\\test\\output.csv")

In [8]:
IMG_SIZE = 224
BATCH_SIZE = 64

In [9]:
train_datagen = ImageDataGenerator(
    brightness_range=[0.4,1.0], 
    horizontal_flip= True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='.\\dataset\\NUOVO\\train',
        x_col="image_id",
        y_col=["x_1","y_1","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory='.\\dataset\\NUOVO\\val',
        x_col="image_id",
        y_col=["x_1","y_1","width","height"],
        class_mode="raw",
        shuffle=True,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 164105 validated image filenames.
Found 18234 validated image filenames.


In [10]:
step_size_train = train_generator.n/train_generator.batch_size
step_size_valid = valid_generator.n/valid_generator.batch_size

In [11]:
vgg_model = VGGFace(model='vgg16', include_top=False, weights='vggface', input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [12]:
vgg_model.trainable = True

In [13]:
vgg_model.summary()

Model: "vggface_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)     

In [14]:
count_conv_layer = 0
print("Vgg layers n°:",len(vgg_model.layers))
window_layers_size = 2
count_layer = 0
for layer in vgg_model.layers: #train only  convolutional layers in the last 50 layers 
    count_layer = count_layer +1
    if (count_layer > (len(vgg_model.layers)-window_layers_size)) and ("conv" in layer.name):
        count_conv_layer = count_conv_layer + 1 
        layer.trainable = True
    else: 
        layer.trainable = False
print("Conv layers in the last",window_layers_size,"layers: ", count_conv_layer)

Vgg layers n°: 19
Conv layers in the last 2 layers:  1


In [15]:
model = keras.Sequential()
model.add(vgg_model)
model.add(Conv2D(filters=1, kernel_size=(1,1), activation="relu"))
model.add(Flatten())

In [16]:
# define last layer output size
input_model_shape = model.input_shape[1]
output_global_pooling_size = model.output_shape[1]

In [17]:
# Rule 1 to choose neurons in hidden layers
rule_1_neurons_number = int((output_global_pooling_size + 4)/2)
# Rule 2 to choose neurons in hidden layers
rule_2_neurons_number = int(output_global_pooling_size/2) + 4
# Rule 2 to choose neurons in hidden layers
alpha = 2
rule_3_neurons_number = int(train_generator.n/(alpha*(4+output_global_pooling_size)))
number_of_hidden_layer = 2
rule_4_neurons_number = [int(round(x/(i+1),0)) for (x,i) in zip([output_global_pooling_size]*number_of_hidden_layer,
                                           range(number_of_hidden_layer))]

In [18]:
print(rule_4_neurons_number[0],rule_4_neurons_number[1])

49 24


In [19]:
model.add(Dense(rule_4_neurons_number[0],activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(rule_4_neurons_number[1],activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4,activation='relu'))

In [20]:
model.compile(optimizer=Adam(learning_rate= 0.0001), loss='mse', metrics=["accuracy"])

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vggface_vgg16 (Model)        (None, 7, 7, 512)         14714688  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 1)           513       
_________________________________________________________________
flatten_1 (Flatten)          (None, 49)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 49)                2450      
_________________________________________________________________
dropout_1 (Dropout)          (None, 49)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                1200      
_________________________________________________________________
dropout_2 (Dropout)          (None, 24)               

In [22]:
model.evaluate_generator(valid_generator, steps=step_size_valid, verbose = 1)

285/284 [==============================] - 51s 178ms/step


[101239.2421875, 0.1698475331068039]

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
mcp_save = ModelCheckpoint('vggface.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='min')

history = model.fit_generator(generator=train_generator, epochs=25, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, callbacks=[mcp_save, earlyStopping, reduce_lr_loss])

Epoch 1/25
2565/2564 [==============================] - 471s 184ms/step - loss: 33804.8952 - accuracy: 0.5598 - val_loss: 47002.0508 - val_accuracy: 0.8098
Epoch 2/25
2565/2564 [==============================] - 470s 183ms/step - loss: 28259.2850 - accuracy: 0.6655 - val_loss: 9136.7207 - val_accuracy: 0.8098
Epoch 3/25
2565/2564 [==============================] - 470s 183ms/step - loss: 26092.9987 - accuracy: 0.7348 - val_loss: 20888.3301 - val_accuracy: 0.8098
Epoch 4/25
2565/2564 [==============================] - 471s 184ms/step - loss: 24918.3279 - accuracy: 0.7824 - val_loss: 12615.6396 - val_accuracy: 0.8098
Epoch 5/25
2565/2564 [==============================] - 470s 183ms/step - loss: 24144.5408 - accuracy: 0.7998 - val_loss: 27994.9492 - val_accuracy: 0.8098

Epoch 00005: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 6/25
1677/2564 [==================>...........] - ETA: 2:27 - loss: 23350.2119 - accuracy: 0.8030

In [ ]:
#print(history.history)

In [ ]:
acc = history.history['precision']
val_acc = history.history['val_precision']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Precision')
plt.plot(val_acc, label='Validation Precision')
plt.legend(loc='lower right')
plt.ylabel('Precision')
plt.ylim([min(plt.ylim()),200])
plt.title('Training and Validation Precision')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,10000])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()


# In[ ]:


print(history.history)

In [ ]:
# In[ ]:

import pandas as pd
dict_data = history.history
df = pd.DataFrame.from_dict(dict_data, dtype=float)
csv_file = "history_resnet.csv"
df.to_csv(csv_file)


# In[ ]:

model_json = model.to_json()
with open("resnet50.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("resnet50.h5")
print("Saved model to disk")

In [ ]:
train_datagen = ImageDataGenerator(
    brightness_range=[0.4,1.0], 
    preprocessing_function=preprocess_input)

test_generator = train_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory='.\\dataset\\test',
        x_col="filename",
        y_col=["x_start","y_start","width","height"],
        class_mode="raw",
        target_size=(300, 300),
        batch_size=128)

step_size_test = test_generator.n/test_generator.batch_size

In [ ]:
import numpy as np

In [ ]:
predictions = model.predict_generator(test_generator, step_size_test, verbose=1)

In [ ]:
print(predictions)

In [ ]:
from PIL import Image, ImageDraw
i = 0

for el in zip(predictions, test_generator.filenames):
    print(el[0], el[1])
    image = Image.open(test_generator.directory + "\\" + el[1])
    draw = ImageDraw.Draw(image)
    draw.rectangle(((el[0][0], el[0][1]), (el[0][0] + el[0][2], el[0][1] + el[0][3])), outline="Red")
    
    image.save('.\\dataset\\test_valuated\\' + el[1])    